### TRƯỜNG ĐẠI HỌC CÔNG NGHIỆP
### THÀNH PHỐ HỒ CHÍ MINH
 
### KHOA Công nghệ Thông tin   
## ĐỀ THI GIỮA KỲ
### Môn thi : Nhập môn dữ liệu lớn 
### Lớp/Lớp học phần:  DHKHDL17A
* Thời gian làm bài: 75 phút (Không kể thời gian phát đề)
* Thí sinh được sử dụng tài liệu và tra cứu tại trang wed
  - https://spark.apache.org/
  - https://stackoverflow.com/
  - - https://learn.microsoft.com/en-us/sql/t-sql
* Thí sinh làm bài và lưu lại với định dạng mssv_hovaten_gk.ipynb. Ví dụ bạn có mã số sinh viên là: 12131411, họ và tên: Hoàng Tiến Anh, thì nộp bài với tên: **12131411_HoangTienAnh_gk.ipynb**
* Thí sinh sử dụng dữ liệu *emails.csv*, điều chỉnh biến 'dataPath' ở cell đầu tiên lại cho đúng với đường dẫn đến file data.
* Hoàn thành tất cả các vị trí có chữ **# YOUR CODE HERE** để hoàn thành yêu cầu của mỗi hàm.
#### LƯU Ý: KHÔNG THAY ĐỔI NHỮNG CHỖ KHÁC

In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame
from pyspark.rdd import RDD
from pyspark.sql.types import Row

sc = SparkSession.builder \
    .appName("Email Data Processing") \
    .getOrCreate()

dataPath = "./email.csv"

In [2]:
import pandas as pd 

df = pd.read_csv(dataPath)
df.head()

,file,message
0,jones-t/sent/5567.,Message-ID: <11629296.1075847544895.JavaMail.e...
1,hayslett-r/all_documents/134.,Message-ID: <11398637.1075844279035.JavaMail.e...
2,taylor-m/all_documents/1477.,Message-ID: <28378907.1075859912982.JavaMail.e...
3,lavorato-j/deleted_items/443.,Message-ID: <30906538.1075861704525.JavaMail.e...
4,farmer-d/logistics/991.,Message-ID: <2344701.1075854128586.JavaMail.ev...


In [3]:
df.isnull().sum()

file       0
message    0
dtype: int64

In [4]:
df.message[0]

"Message-ID: <11629296.1075847544895.JavaMail.evans@thyme>\nDate: Thu, 1 Feb 2001 08:00:00 -0800 (PST)\nFrom: tana.jones@enron.com\nTo: bob.bowen@enron.com, laurel.adams@enron.com, andrea.guillen@enron.com, \n\tlarry.hunter@enron.com, gordon.heaney@enron.com, \n\tjanie.aguayo@enron.com, derek.bailey@enron.com, \n\tdiane.anderson@enron.com, kevin.meredith@enron.com, \n\tscott.tackett@enron.com, celeste.cisneros@enron.com, \n\ttiffany.williams@enron.com, kim.theriot@enron.com, \n\tdianne.seib@enron.com, melissa.murphy@enron.com, \n\twilliam.bradford@enron.com, debbie.brackett@enron.com, \n\ttanya.rohauer@enron.com, brant.reves@enron.com, tracy.ngo@enron.com, \n\tlesli.campbell@enron.com, veronica.espinoza@enron.com, \n\tpaul.radous@enron.com, wendi.lebrocq@enron.com, \n\trussell.diamond@enron.com, tom.moran@enron.com, \n\tedward.sacks@enron.com, nidia.mendoza@enron.com, \n\tbradley.diebner@enron.com, wendy.conwell@enron.com, \n\tdarren.vanek@enron.com, rudwell.johnson@enron.com, \n\tbren

In [5]:
from functools import reduce # thêm
from pyspark.sql.functions import col # thêm

In [6]:
#0.5
def loadAndProcessCsv(filePath: str, spark: SparkSession) -> DataFrame:
    '''
    This function loads a CSV file into a Spark DataFrame, caches it, 
    drops rows with null values, and prints the schema. (using option when read to keep format - header, columns)
    
    Args:
        file_path (str): Path to the CSV file.
        spark (SparkSession): Active Spark session.
        
    Returns:
        DataFrame: Processed DataFrame.
    '''

    data = None
    # YOUR CODE HERE
    data = spark.read \
        .format("csv") \
        .option("header","true") \
        .option("inferSchema","true") \
        .option("samplingRatio",.01) \
        .option("multiLine","true") \
        .option("escape", '"') \
        .option("quote", '"') \
        .option("sep", ",") \
        .load(filePath)
    
    # 1. **`header`**: `"true"` - Dòng đầu là tên cột.
    # 2. **`inferSchema`**: `"true"` - Tự suy luận kiểu dữ liệu.
    # 3. **`samplingRatio`**: `.01` - Lấy mẫu 1% để suy luận kiểu thay vì lấy hết thì rất lâu.
    # 4. **`multiLine`**: `"true"` - Hỗ trợ nhiều dòng trong một ô.
    # 5. **`escape`**: `'"'` - Ký tự thoát cho dấu ngoặc kép.
    # 6. **`quote`**: `'"'` - Ký tự bao quanh chuỗi (dấu ngoặc kép).
    # 7. **`sep`**: `","` - Dấu phân cách giữa các cột (dấu phẩy).

    # Lọc các hàng có giá trị null trong bất kỳ cột nào
    data.filter(
        reduce(lambda a, b: a | b, (col(c).isNull() for c in data.columns))
    ).show()

    data = data.dropna()
    
    data.cache()
    
    print("------Schema------")
    data.printSchema()
    
    return data

In [7]:
data = loadAndProcessCsv(dataPath,sc)

+----+-------+
|file|message|
+----+-------+
+----+-------+

------Schema------
root
 |-- file: string (nullable = true)
 |-- message: string (nullable = true)



In [8]:
num_partitions = data.rdd.getNumPartitions()
print("Số lượng partition:", num_partitions)

Số lượng partition: 1


In [9]:
data.columns

['file', 'message']

In [10]:
data.show()

+--------------------+--------------------+
|                file|             message|
+--------------------+--------------------+
|  jones-t/sent/5567.|Message-ID: <1162...|
|hayslett-r/all_do...|Message-ID: <1139...|
|taylor-m/all_docu...|Message-ID: <2837...|
|lavorato-j/delete...|Message-ID: <3090...|
|farmer-d/logistic...|Message-ID: <2344...|
|donohoe-t/all_doc...|Message-ID: <2090...|
|gang-l/deleted_it...|Message-ID: <2583...|
|guzman-m/discussi...|Message-ID: <1170...|
|kean-s/all_docume...|Message-ID: <1471...|
|meyers-a/deleted_...|Message-ID: <2596...|
|keavey-p/discussi...|Message-ID: <2588...|
|maggi-m/deleted_i...|Message-ID: <1966...|
|campbell-l/discus...|Message-ID: <1376...|
|jones-t/all_docum...|Message-ID: <1024...|
|mann-k/_sent_mail...|Message-ID: <1871...|
|haedicke-m/all_do...|Message-ID: <8046...|
|whalley-l/discuss...|Message-ID: <2996...|
|campbell-l/notes_...|Message-ID: <2743...|
|kean-s/discussion...|Message-ID: <5027...|
|love-p/_sent_mail...|Message-ID

In [11]:
from pyspark.sql.functions import *
data.select(count("*").alias("total_rows"),
            countDistinct("file").alias("file_unique")).show()

+----------+-----------+
|total_rows|file_unique|
+----------+-----------+
|     20000|      20000|
+----------+-----------+



In [12]:
data.selectExpr("count(*) as total_rows",
               "count(distinct (file)) as fine_unique").show()

+----------+-----------+
|total_rows|fine_unique|
+----------+-----------+
|     20000|      20000|
+----------+-----------+



In [13]:
data.createOrReplaceTempView("emails")

In [14]:
sc.sql("select count(*) as total_rows, count(distinct(file)) as file_unique from emails").show()

+----------+-----------+
|total_rows|file_unique|
+----------+-----------+
|     20000|      20000|
+----------+-----------+



In [15]:
# 0.5đ
def createRDD(data: DataFrame)->RDD[Row]:
    '''
    This function converts a Spark DataFrame into an RDD of Rows.
    
    Args:
        data (DataFrame): Input DataFrame containing data loaded by Spark.
        
    Returns:
        RDD[Row]: An RDD containing all Rows from the input DataFrame.
    '''
    outRDD = None

    # YOUR CODE HERE
    outRDD = data.rdd
    return outRDD

In [16]:
emailRDD = createRDD(data)
assert isinstance(emailRDD, RDD), "createRDD() does not return the correct data type (RDD)"
assert isinstance(emailRDD.first(),Row), "createRDD() contains elements that are not of type Row"

In [17]:
emailRDD.first()

Row(file='jones-t/sent/5567.', message="Message-ID: <11629296.1075847544895.JavaMail.evans@thyme>\nDate: Thu, 1 Feb 2001 08:00:00 -0800 (PST)\nFrom: tana.jones@enron.com\nTo: bob.bowen@enron.com, laurel.adams@enron.com, andrea.guillen@enron.com, \n\tlarry.hunter@enron.com, gordon.heaney@enron.com, \n\tjanie.aguayo@enron.com, derek.bailey@enron.com, \n\tdiane.anderson@enron.com, kevin.meredith@enron.com, \n\tscott.tackett@enron.com, celeste.cisneros@enron.com, \n\ttiffany.williams@enron.com, kim.theriot@enron.com, \n\tdianne.seib@enron.com, melissa.murphy@enron.com, \n\twilliam.bradford@enron.com, debbie.brackett@enron.com, \n\ttanya.rohauer@enron.com, brant.reves@enron.com, tracy.ngo@enron.com, \n\tlesli.campbell@enron.com, veronica.espinoza@enron.com, \n\tpaul.radous@enron.com, wendi.lebrocq@enron.com, \n\trussell.diamond@enron.com, tom.moran@enron.com, \n\tedward.sacks@enron.com, nidia.mendoza@enron.com, \n\tbradley.diebner@enron.com, wendy.conwell@enron.com, \n\tdarren.vanek@enron.c

In [18]:
import email
from typing import List, Optional
from pyspark.sql.types import Row

def splitEmailAddresses(emailString: str) -> List[Optional[str]]:
    '''
    The function splits a comma-separated string of email addresses into a unique list.
    
    Args:
        emailString: A string containing email addresses separated by commas.
        
    Returns:
        A list of unique email addresses.
    '''
    if emailString:
        addresses = emailString.split(',')
        uniqueAddresses = list(frozenset(map(lambda x: x.strip(), addresses)))
        return uniqueAddresses
    return []

def extractEmailDetailsFromRawText(rawEmail: str) -> Row:
    '''
    The function extracts relevant details from a raw email message string.
    
    Args:
        rawEmail: A string representing the raw email message.
        
    Returns:
        A Row object containing the extracted email details.
    '''
    emailMessage = email.message_from_string(rawEmail)
    emailContentParts = []
    for part in emailMessage.walk():
        if part.get_content_type() == 'text/plain':
            emailContentParts.append(part.get_payload())

    emailContent = ''.join(emailContentParts)

    fromAddresses = splitEmailAddresses(emailMessage.get("From"))
    toAddresses = splitEmailAddresses(emailMessage.get("To"))
    ccEmail = splitEmailAddresses(emailMessage.get("Cc"))
    return Row(
        Date=emailMessage.get("Date"),
        From=fromAddresses, 
        To=toAddresses, 
        Subject=emailMessage.get("Subject"), 
        CC=ccEmail, 
        Content=emailContent
    )

# Extract structured email details from the first email message
firstEmailData = data.first()
structuredEmail = extractEmailDetailsFromRawText(firstEmailData.message)
structuredEmail

Row(Date='Thu, 1 Feb 2001 08:00:00 -0800 (PST)', From=['tana.jones@enron.com'], To=['larry.hunter@enron.com', 'tanya.rohauer@enron.com', 'kim.theriot@enron.com', 'mary.cook@enron.com', 'darren.vanek@enron.com', 'frank.sayre@enron.com', 'kevin.meredith@enron.com', 'robert.bruce@enron.com', 'nidia.mendoza@enron.com', 'tom.moran@enron.com', 'susan.bailey@enron.com', 'bob.bowen@enron.com', 'cheryl.johnson@enron.com', 'frank.davis@enron.com', 'bradley.diebner@enron.com', 'stephanie.panus@enron.com', 'derek.bailey@enron.com', 'paul.radous@enron.com', 'francisco.leite@enron.com', 'adnan.patel@enron.com', 'bernice.rodriguez@enron.com', 'lisa.lees@enron.com', 'cheryl.nelson@enron.com', 'scott.tackett@enron.com', 'edward.sacks@enron.com', 'debbie.brackett@enron.com', 'sara.shackleton@enron.com', 'lynn.shivers@enron.com', 'dianne.seib@enron.com', 'samuel.schott@enron.com', 'georgi.landau@enron.com', 'tiffany.williams@enron.com', 'anthony.campos@enron.com', 'mary.gosnell@enron.com', 'carrie.southa

In [19]:
emailRDD.first()

Row(file='jones-t/sent/5567.', message="Message-ID: <11629296.1075847544895.JavaMail.evans@thyme>\nDate: Thu, 1 Feb 2001 08:00:00 -0800 (PST)\nFrom: tana.jones@enron.com\nTo: bob.bowen@enron.com, laurel.adams@enron.com, andrea.guillen@enron.com, \n\tlarry.hunter@enron.com, gordon.heaney@enron.com, \n\tjanie.aguayo@enron.com, derek.bailey@enron.com, \n\tdiane.anderson@enron.com, kevin.meredith@enron.com, \n\tscott.tackett@enron.com, celeste.cisneros@enron.com, \n\ttiffany.williams@enron.com, kim.theriot@enron.com, \n\tdianne.seib@enron.com, melissa.murphy@enron.com, \n\twilliam.bradford@enron.com, debbie.brackett@enron.com, \n\ttanya.rohauer@enron.com, brant.reves@enron.com, tracy.ngo@enron.com, \n\tlesli.campbell@enron.com, veronica.espinoza@enron.com, \n\tpaul.radous@enron.com, wendi.lebrocq@enron.com, \n\trussell.diamond@enron.com, tom.moran@enron.com, \n\tedward.sacks@enron.com, nidia.mendoza@enron.com, \n\tbradley.diebner@enron.com, wendy.conwell@enron.com, \n\tdarren.vanek@enron.c

In [20]:
#0.5đ
def createStructuredEmailRDD(emailRDD: RDD[Row]) -> RDD[Row]:
    '''
    The function takes an RDD of email messages and converts it into a new RDD containing structured email details.
    
    Args:
        emailRDD: An RDD where each Row contains an email message in raw text format.
        
    Returns:
        A new RDD where each element is a Row with structured email details such as Date, From, To, Subject, CC, and Content.
    '''
    structuredEmailRDD = None
    # YOUR CODE HERE
    structuredEmailRDD = emailRDD.map(lambda x: extractEmailDetailsFromRawText(x.message))
    return structuredEmailRDD

In [21]:
structuredEmailRDD = createStructuredEmailRDD(emailRDD)
assert isinstance(structuredEmailRDD, RDD), "createStructuredEmailRDD() doesn't return an RDD"
assert isinstance(structuredEmailRDD.first(), Row), "createStructuredEmailRDD() elements are not of type Row"

In [22]:
structuredEmailRDD.take(1)  # Lấy 10 phần tử đầu tiên

[Row(Date='Thu, 1 Feb 2001 08:00:00 -0800 (PST)', From=['tana.jones@enron.com'], To=['carol.clair@enron.com', 'russell.diamond@enron.com', 'derek.bailey@enron.com', 'cheryl.nelson@enron.com', 'adnan.patel@enron.com', 'tom.moran@enron.com', 'darren.vanek@enron.com', 'diane.ellstrom@enron.com', 'sara.shackleton@enron.com', 'william.bradford@enron.com', 'edward.sacks@enron.com', 'frank.davis@enron.com', 'nidia.mendoza@enron.com', 'bob.bowen@enron.com', 'andrea.guillen@enron.com', 'kevin.meredith@enron.com', 'celeste.cisneros@enron.com', 'carrie.southard@enron.com', 'diane.anderson@enron.com', 'melissa.murphy@enron.com', 'francisco.leite@enron.com', 'mary.gosnell@enron.com', 'wendi.lebrocq@enron.com', 'mark.taylor@enron.com', 'kim.theriot@enron.com', 'stacey.richardson@enron.com', 'gordon.heaney@enron.com', 'dianne.seib@enron.com', 'dale.neuner@enron.com', 'debbie.brackett@enron.com', 'georgi.landau@enron.com', 'cheryl.johnson@enron.com', 'susan.bailey@enron.com', 'mary.cook@enron.com', 'b

In [23]:
structuredEmailRDD.first().CC

['larry.gagliardi@enron.com',
 'matthew.dawson@enron.com',
 "denis.o'connell@enron.com"]

In [24]:
#1.
def countNumberEmail(structuredEmailRDD: RDD[Row], k: int)->int:
    '''
    The function counts the number of emails with more than `k` email addresses in the 'To' field.
    
    Args:
    - structuredEmailRDD: RDD of Row objects, each containing an email's structured data.
    - k: The threshold for the number of emails in the 'To' field.
    
    Returns:
    - int: The count of emails with more than `k` email addresses in the 'To' field.
    '''
    count = -1
    # YOUR CODE HERE
    # count = 0
    # for element in structuredEmailRDD.collect():
    #     if len(element.To) > k :
    #         count+=1
    count = structuredEmailRDD.filter(lambda row : len(row.To) > k).count()
    
    return count

In [25]:
countNumberEmail(structuredEmailRDD,40)

647

In [26]:
# 1đ
def countUniqueEmailDomains(structuredEmailRDD: RDD[Row], k) -> int:
    '''
    This function counts the number of unique email domains in the "CC" field using `map` and `reduce`.
    
    Args:
    - structuredEmailRDD: An RDD containing Row objects, each representing an email's structured data.
    
    Returns:
    - dict: A dictionary showing the count of emails from each unique domain in the "CC" field.
      Example:
        If k = 3, the result might look like:
        {
          'enron.com': 16452,
          'aol.com': 122,
          'hotmail.com': 101
        }
    '''
    results = {}
    # YOUR CODE HERE
    # list_cc = []
    # for element in structuredEmailRDD.collect():
    #     list_cc += element.CC
    ccRDD = structuredEmailRDD.flatMap(lambda row : row.CC) 

    # list_cc = sc.sparkContext.parallelize(list_cc)
    ccRDD = ccRDD.map(lambda x: (x.split("@")[-1],1))
    domainCountRDD = ccRDD.reduceByKey(lambda a,b: a+b)
    
    # for element in domainCountRDD.collect():
    #     results[element[0]] = element[1]
    domainCountRDD = domainCountRDD.sortBy(lambda x : x[1],ascending=False)
    results = domainCountRDD.collectAsMap()
    # results = dict(sorted(results.items(), key=lambda item: item[1], reverse=True)[:k])
    results = dict(list(results.items())[:k])
    return results
    

In [27]:
countUniqueEmailDomains(structuredEmailRDD, 10)

{'enron.com': 18864,
 'aol.com': 92,
 'hotmail.com': 79,
 'duke-energy.com': 68,
 'haas.berkeley.edu': 50,
 'caiso.com': 47,
 'enron.com>': 44,
 'yahoo.com': 44,
 'iepa.com': 37,
 'akllp.com': 35}

In [28]:
# 0.5đ ;
def countEmailsBySubject(structuredEmailRDD: RDD[Row], subjectContent: str) -> int:
    '''
    The function count num emails the dataset to the subject have included "subjectContent", case insensitive

    Args:
    - structuredEmailRDD: RDD of Row objects containing an email's structured data.
    - subSubject: The content of the subject to filter by.

    Returns:
    - numEmails: Subject include content have substring subjectContent
    '''
    numEmails =  -1
    # YOUR CODE HERE
    # numEmails = structuredEmailRDD.filter(lambda x: x.Subject.strip() != '' and x.Subject.strip() != '(no subject)').count()
    numEmails = structuredEmailRDD.filter(lambda x: subjectContent.strip().lower() in x.Subject.strip().lower() ).count()
    return numEmails

In [29]:
structuredEmailRDD.first().asDict().keys()

dict_keys(['Date', 'From', 'To', 'Subject', 'CC', 'Content'])

In [30]:
structuredEmailRDD.first().Subject

'Deutsche Bank AG'

In [31]:
df_struct = structuredEmailRDD.toDF()

In [32]:
df_struct.groupBy("Subject").count().orderBy("count",ascending = False).show()

+--------------------+-----+
|             Subject|count|
+--------------------+-----+
|                    |  760|
|                 Re:|  257|
|                 RE:|  239|
|Demand Ken Lay Do...|   45|
|Schedule Crawler:...|   45|
|Schedule Crawler:...|   43|
|                 FW:|   39|
|      Enron Mentions|   24|
|       Energy Issues|   20|
|        (no subject)|   19|
|Williams Energy N...|   18|
|Organizational An...|   16|
|  EnTouch Newsletter|   15|
|RIGZONE Industry ...|   15|
|Organization Anno...|   13|
|Organizational Ch...|   11|
|WARNING:  Your ma...|   11|
|Mid-Year 2001 Per...|   11|
|       Power Indices|   10|
|      Attached Files|   10|
+--------------------+-----+
only showing top 20 rows



In [33]:
countEmailsBySubject(structuredEmailRDD, "bank")

78

In [34]:
# 1.5d
def getTopKFrequentWordsInContentBySender(structuredEmailRDD: RDD[Row], sender: str, k: int) -> dict:
    '''
    This function filtered emails in the dataset sent by a specific sender's email,
    and then returns the top k most frequent words found in the content of those filtered emails.

    Args:
    - structuredEmailRDD: RDD of Row objects, where each Row represents an email with structured data, such as subject and content.
    - sender: sender's email which needs to count.
    - k: The number of most frequent words to return.

    Returns:
    - topKWordsDict: A dictionary containing the top k most frequent words found in the content of filtered emails.
                     The keys are the words, and the values are their frequencies, representing how often they appear in the content of the filtered emails.
                    example:
                    {'to': 12,
                     'the': 9,
                     'your': 8,
                     'a': 5,
                     'is': 4,
                     'survey': 4,
                     'and': 4,
                     'you': 4,
                     'of': 3,
                     'very': 3}
    '''
    topKWordsDict = {}
    # YOUR CODE HERE
    senderRDD = structuredEmailRDD.filter(lambda row : row.From[0].strip() == sender)
    content_senderRDD = senderRDD.flatMap(lambda row : row.Content.split())
    content_senderRDD = content_senderRDD.filter(lambda x : x != '')

    wordPairs = content_senderRDD.map(lambda x : (x,1))
    wordcounts = wordPairs.reduceByKey(lambda a,b : a+b)

    dictWordCounts = wordcounts.sortBy(lambda x: x[1], ascending=False).collectAsMap()
    topKWordsDict = dict(list(dictWordCounts.items())[:k])
    # dictWordCounts = wordcounts.collectAsMap()
    # topKWordsDict = dict(sorted(dictWordCounts.items(),key=lambda item: item[1], reverse=True)[:k])
    
    # raise NotImplementedError()
    return topKWordsDict

In [35]:
# structuredEmailRDD.first().From[0],structuredEmailRDD.first().Content

In [36]:
getTopKFrequentWordsInContentBySender(structuredEmailRDD, "tana.jones@enron.com", 10)

{'the': 1388,
 'to': 953,
 'and': 697,
 'of': 637,
 'I': 485,
 'a': 484,
 'on': 439,
 'for': 424,
 'you': 409,
 'Tana': 388}

In [37]:
# SQL Query
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, size, to_date, year, month, weekofyear
sc.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
# Convert RDD to DataFrame
dfEmais = structuredEmailRDD.toDF()
dfEmais.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                Date|                From|                  To|             Subject|                  CC|             Content|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Thu, 1 Feb 2001 0...|[tana.jones@enron...|[carol.clair@enro...|    Deutsche Bank AG|[larry.gagliardi@...|We have received ...|
|Wed, 19 Jul 2000 ...|[patrick.cagney@e...|                  []|Report Council Mt...|[michael.sullivan...|Reporting Council...|
|Fri, 3 Mar 2000 0...|[jenny.helton@enr...|[david.forster@en...|SO2 Enron EAuctio...|                  []|As promised by no...|
|Tue, 13 Nov 2001 ...|[mailing1@mail.mo...|[lavorato@enron.com]|Increase Your Inc...|                  []|Master the secret...|
|Wed, 29 Mar 2000 ...|[pat.clynes@enron...|[kim.melodick@enr...| Re: resume attached|[rita.wynne@enron..

In [38]:
dfEmais = dfEmais.withColumn('DateTime', to_date(col('Date'), "EEE, d MMM yyyy HH:mm:ss Z"))
dfEmais = dfEmais.withColumn('Num_To', size(col('To')))
dfEmais = dfEmais.withColumn('Num_CC', size(col('CC')))

In [39]:
sqlContext = SQLContext(sc)
tableName = "Emails"
dfEmais.createOrReplaceTempView(tableName)

/usr/local/spark/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [40]:
# 0.5đ
def getTopKRowsBySQL(sqlContext: SQLContext, tableName: str, k: int) -> DataFrame:
    '''
    This function queries the first k rows from a given table using SQLContext.

    Args:
    - sqlContext: An SQLContext object that provides the environment to run SQL queries on structured data.
    - tableName: The name of the table from which the rows will be selected.
    - k: The number of rows to return. It must be a positive integer.

    Returns:
    - result: A DataFrame containing the first k rows of data from the specified table.
                If the table contains fewer than k rows, the DataFrame will contain all available rows.
    '''
    # YOUR CODE HERE
    if k <= 0:
        raise ValueError("k must be a positive integer")
    result = sqlContext.sql(f"select * from {tableName} LIMIT {k}")
    return result


def getTopKRowsByDFOperations(dataFrame: DataFrame, k: int) -> DataFrame:
    '''
    This function queries the first k rows from a given table using SQLContext.

    Args:
    - dataFrame: An DataFrame data
    - tableName: The name of the table from which the rows will be selected.
    - k: The number of rows to return. It must be a positive integer.

    Returns:
    - resultDF: A DataFrame containing the first k rows of data from the specified table.
                If the table contains fewer than k rows, the DataFrame will contain all available rows.
    '''
    # YOUR CODE HERE
    if k <= 0:
        raise ValueError("k must be a positive integer")
    result = dataFrame.limit(k)
    return result

In [41]:
getTopKRowsBySQL(sqlContext,tableName,1).show()

+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+----------+------+------+
|                Date|                From|                  To|         Subject|                  CC|             Content|  DateTime|Num_To|Num_CC|
+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+----------+------+------+
|Thu, 1 Feb 2001 0...|[tana.jones@enron...|[carol.clair@enro...|Deutsche Bank AG|[larry.gagliardi@...|We have received ...|2001-02-01|    64|     3|
+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+----------+------+------+



In [42]:
getTopKRowsByDFOperations(dfEmais,1).show()

+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+----------+------+------+
|                Date|                From|                  To|         Subject|                  CC|             Content|  DateTime|Num_To|Num_CC|
+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+----------+------+------+
|Thu, 1 Feb 2001 0...|[tana.jones@enron...|[carol.clair@enro...|Deutsche Bank AG|[larry.gagliardi@...|We have received ...|2001-02-01|    64|     3|
+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+----------+------+------+



In [43]:
# 1đ, 6.5đ
def countEmailsWithToGreaterThanKBySQL(sqlContext: SQLContext, tableName: str, k: int) -> int:
    '''
    This function uses SQL to count the number of emails with more than k email addresses in the To field.

    Args:
    - sqlContext: An SQLContext object that provides the environment to run SQL queries on structured data.
    - tableName: The name of the table that contains the email data.
    - k: The threshold number of email addresses in the To field.

    Returns:
    - count: An integer representing the number of emails where the To field has more than k email addresses.
    '''
    # YOUR CODE HERE
    # raise NotImplementedError()
    count = sqlContext.sql(f"select count(*) as email_count from {tableName} where `Num_To` > {k};").collect()[0]['email_count']
    return count
def countEmailsWithToGreaterThanKByDFOperations(dataFrame: DataFrame, k: int) -> int:
    '''
    This function counts the number of emails with more than k email addresses in the To field using DataFrame operations.

    Args:
    - dataFrame: A DataFrame containing the email data.
    - k: The threshold number of email addresses in the To field.

    Returns:
    - count: An integer representing the number of emails where the To field has more than k email addresses.
    '''
    # YOUR CODE HERE
    # raise NotImplementedError()
    # result = dataFrame.rdd.filter(lambda x : (x['Num_To']) > k).count()
    # result = dataFrame.filter(dataFrame['Num_To'] > k).count()
    result = dataFrame.filter(col('Num_To') > k).count()
    return result


In [44]:
countEmailsWithToGreaterThanKBySQL(sqlContext,tableName,50)

525

In [45]:
countEmailsWithToGreaterThanKByDFOperations(dfEmais,50)

525

In [46]:
def countEmailsWithMoreToThanCCInJanuaryBySQL(sqlContext, tableName):
    # "đếm số lượng email có số lượng địa chỉ trong trường "To" nhiều hơn số lượng địa chỉ trong trường "CC" trong tháng Giêng (January)"
    query = f"""
    SELECT COUNT(*) FROM {tableName}
    WHERE Num_To > Num_CC and month(DateTime) = 1;
    """
    count = sqlContext.sql(query).collect()[0][0]
    
    # count = sqlContext.sql(f"select count(*) from {tableName} where Num_To > Num_CC and month(DateTime) = 1").collect()[0][0]
    return count

def countEmailsWithMoreToThanCCInJanuaryByDFOperations(dataFrame: DataFrame, monthValue: int):
    #  đếm số lượng email có số lượng địa chỉ trong trường "To" lớn hơn số lượng địa chỉ trong trường "CC" .
    # count = dataFrame.filter((dataFrame.Num_To > dataFrame.Num_CC) & (month(dataFrame.DateTime) == monthValue)).count()
    count = dataFrame.filter((month(col("DateTime")) == monthValue) & (col("Num_To") > col("Num_CC"))).count()

    return count

In [47]:
countEmailsWithMoreToThanCCInJanuaryBySQL(sqlContext, tableName)

1567

In [48]:
countEmailsWithMoreToThanCCInJanuaryByDFOperations(dfEmais,1)

1567

In [49]:
# 1đ, 8.5
def countEmailsInMonthBySQL(sqlContext: SQLContext, tableName: str, monthValue: int) -> int:
    '''
    This function calculates the number of emails sent in a given year by performing a group by operation on the DateTime column using SQL.

    Args:
    - sqlContext: An SQLContext object that provides the environment to run SQL queries on structured data.
    - tableName: The name of the table that contains the email data.
    - monthValue: The specific month to filter the emails by (1-12).

    Returns:
    - email_count: An integer representing the number of emails sent in the given date.
    '''
    # YOUR CODE HERE
    # raise NotImplementedError()
    query = f"""
    SELECT COUNT(*) FROM {tableName}
    WHERE MONTH(DateTime) = {monthValue}
    """
    emailCount = sqlContext.sql(query).collect()[0][0]
    return emailCount

def countEmailsInMonthByDFOperations(dataFrame: DataFrame, monthValue: int) -> int:
    '''
    This function calculates the number of emails sent in a given month by performing a filter operation on the DateTime column using DataFrame operations.

    Args:
    - dataFrame: A DataFrame containing the email data.
    - monthValue: The specific date to filter the emails by.

    Returns:
    - email_count: An integer representing the number of emails sent in the given monthValue.
    '''
    assert isinstance(dataFrame, DataFrame), f"dataFrame is not the correct data type, expected DataFrame but got {type(dataFrame)}"
    assert isinstance(monthValue, int) and monthValue > 0 and monthValue <= 12, f"date must be a positive integer, but got {monthValue}"
    # YOUR CODE HERE
    # raise NotImplementedError()
    result = dataFrame.filter(month(col("DateTime")) == monthValue).count()
    return result


In [50]:
countEmailsInMonthBySQL(sqlContext,tableName,5)

1826

In [51]:
countEmailsInMonthByDFOperations(dfEmais,5)

1826

In [52]:
# 1đ
def countEmailsPerWeekBySQL(sqlContext: SQLContext, tableName: str) -> DataFrame:
    '''
    This function calculates the number of emails sent per Week by performing a group by operation on the DateTime column 
    and sorts the result in descending order by the number of emails.

    Args:
    - sqlContext: An SQLContext object that provides the environment to run SQL queries on structured data.
    - tableName: The name of the table that contains the email data.

    Returns:
    - resultDF: A DataFrame containing the number of emails sent per Week, sorted in descending order by num week.
                With schema:
                    root
                     |-- week: integer (nullable = true)
                     |-- count: long (nullable = false)
                    
    '''
    assert isinstance(sqlContext, SQLContext), f"sqlContext is not the correct data type, expected SQLContext but got {type(sqlContext)}"
    assert isinstance(tableName, str), f"tableName is not the correct data type, expected string but got {type(tableName)}"
    # YOUR CODE HERE
    # raise NotImplementedError()
    query = f"""
    SELECT WEEKOFYEAR(DateTime) AS week, COUNT(*) AS emails_count FROM {tableName}
    GROUP BY WEEKOFYEAR(DateTime) 
    ORDER BY week DESC;
    """
    result = sqlContext.sql(query)
    return result


def countEmailsPerWeekByDFOperations(dataFrame: DataFrame) -> DataFrame:
    '''
    This function calculates the number of emails sent per Week by performing a group by operation on the DateTime column 
    and sorts the result in descending order by the number of emails.

    Args:
    - dataFrame: A DataFrame containing the email data.

    Returns:
    - resultDF: A DataFrame containing the number of emails sent per Week, sorted in descending order by num week.
                With schema:
                    root
                     |-- week: integer (nullable = true)
                     |-- count: long (nullable = false)
    '''
    assert isinstance(dataFrame, DataFrame), f"dataFrame is not the correct data type, expected DataFrame but got {type(dataFrame)}"
    # YOUR CODE HERE
    # raise NotImplementedError()
    result = dataFrame.groupBy(weekofyear('DateTime').alias("week")).count().alias("emails_count").orderBy("week",ascending = False)
    return result

In [53]:
countEmailsPerWeekBySQL(sqlContext,tableName).show()

+----+------------+
|week|emails_count|
+----+------------+
|  52|         193|
|  51|         338|
|  50|         616|
|  49|         491|
|  48|         700|
|  47|         518|
|  46|         628|
|  45|         481|
|  44|         503|
|  43|         768|
|  42|         576|
|  41|         405|
|  40|         349|
|  39|         350|
|  38|         285|
|  37|         311|
|  36|         227|
|  35|         307|
|  34|         285|
|  33|         225|
+----+------------+
only showing top 20 rows



In [54]:
countEmailsPerWeekByDFOperations(dfEmais).show()

+----+-----+
|week|count|
+----+-----+
|  52|  193|
|  51|  338|
|  50|  616|
|  49|  491|
|  48|  700|
|  47|  518|
|  46|  628|
|  45|  481|
|  44|  503|
|  43|  768|
|  42|  576|
|  41|  405|
|  40|  349|
|  39|  350|
|  38|  285|
|  37|  311|
|  36|  227|
|  35|  307|
|  34|  285|
|  33|  225|
+----+-----+
only showing top 20 rows

